# XCP

> flashing interface for XCP
> For now, only support XCP on CAN via Kvaser CAN interface

In [1]:
#| default_exp xcp

In [108]:
#| hide
from nbdev.showdoc import show_doc
from fastcore.test import *

In [145]:
#| export
import os
import git
import argparse
from InquirerPy import inquirer
from InquirerPy.validator import EmptyInputValidator
from InquirerPy.base.control import Choice
from pydantic import ValidationError
from pprint import pprint, PrettyPrinter

In [148]:
#| export
import subprocess
from multiprocessing import Manager
from multiprocessing.managers import DictProxy
import can
import cantools
from cantools.database import Message as MessagerTpl
from cantools.database.can.database import Database

In [64]:
#| export
import pandas as pd
import numpy as np
import struct

In [5]:
#| export 
from candycan.a2l import (
    list_of_strings,
    XCPCalib,
    XCPData,
    XCPConfig,
    Get_XCPCalib_From_XCPJSon,
    Generate_Init_XCPData_From_A2L,
)


'/home/n/devel/anting_remote/eos/tspace/candycan'


In [6]:
#| export
pp = PrettyPrinter(indent=4, width=80, compact=True)

In [7]:
#| export 
repo = git.Repo("./", search_parent_directories=True)  # get the Repo object of tspace
if os.path.basename(repo.working_dir) != "candycan":  # I'm in the parent repo!
    repo = repo.submodule("candycan").module()
pprint(repo.working_dir)

'/home/n/devel/anting_remote/eos/tspace/candycan'


In [16]:
#| export
def get_argparser() -> argparse.ArgumentParser:
	"""Summary
	Get argument parser for command line arguments

	Returns:
		argparse.ArgumentParser: _description_
	"""
	parser = argparse.ArgumentParser(description='XCP Processing')

	parser.add_argument(
		'--protocol',
		type=str,
		choices=['ccp', 'xcp'],
		default='ccp',
		help='Protocol to use: ccp/xcp',
	)

	parser.add_argument(
		'--download',
		default=False,
		help='Download or upload: default is download(host->target)',
		action='store_true',
	)

	parser.add_argument(
		'--diff_flashing',
		default=True,
		help='use differential flashing',
		action='store_false',
	)

	parser.add_argument(
		'--a2l', 
		type=str,
        default=repo.working_dir+'/res/VBU_AI.json',
		help='a2l json file path')
		
	parser.add_argument(
		"--node-path",
		type=str,
		default=r"/PROJECT/MODULE[]",
		help="node path to search for calibration parameters",
	)

	parser.add_argument(
		"--leaves",
		type=list_of_strings,
		default=r"TQD_trqTrqSetNormal_MAP_v, " 
				r"VBU_L045A_CWP_05_09T_AImode_CM_single, " 
				r"Lookup2D_FLOAT32_IEEE, " 
				r"Lookup2D_X_FLOAT32_IEEE, " 
				r"Scalar_FLOAT32_IEEE, " 
				r"TQD_vVehSpd, "
				r"TQD_vSgndSpd_MAP_y, "
				r"TQD_pctAccPedPosFlt, "
				r"TQD_pctAccPdl_MAP_x",
			help="leaf nodes to search for",
	)

	parser.add_argument(
		'--channel', 
		type=int,
		default=3,
		help='CAN channel for flashing')
		
	parser.add_argument(
		'--download_id', 
		type=int,
		default=630,
		help='CAN message ID for downloading')
		
	parser.add_argument(
		'--upload_id', 
		type=int,
		default=631,
		help='CAN message ID for downloading')
		
	parser.add_argument(
		'--input', 
		type=str,
        default=repo.working_dir+'/res/download.json',
		help='Input file path')
	
	parser.add_argument(
		'--output', 
		type=str, 
        default=repo.working_dir+'/res/output.json',
		help='Output file path')
	return parser

In [13]:
show_doc(get_argparser)

---

[source](https://github.com/Binjian/candycan/blob/main/candycan/xcp.py#L37){target="_blank" style="float:right; font-size:smaller"}

### get_argparser

>      get_argparser ()

Summary
Get argument parser for command line arguments

Returns:
        argparse.ArgumentParser: _description_

In [24]:
parser = get_argparser()
args = parser.parse_args(
    args=[
        '--protocol', 'xcp',
        '--download',
        '--a2l', repo.working_dir+'/res/VBU_AI.json',
        '--node-path', r'/PROJECT/MODULE[]',
        '--leaves', r'TQD_trqTrqSetNormal_MAP_v, VBU_L045A_CWP_05_09T_AImode_CM_single, Lookup2D_FLOAT32_IEEE, Lookup2D_X_FLOAT32_IEEE, Scalar_FLOAT32_IEEE, TQD_vVehSpd, TQD_vSgndSpd_MAP_y, TQD_pctAccPedPosFlt, TQD_pctAccPdl_MAP_x',
        '--channel', '3',
        '--download_id', '630',
        '--upload_id', '631',
        '--input', repo.working_dir+'/res/download.json',
        '--output', repo.working_dir+'/res/output.json',
    ]
)

Namespace(protocol='xcp', download=True, diff_flashing=True, a2l='/home/n/devel/anting_remote/eos/tspace/candycan/res/VBU_AI.json', node_path='/PROJECT/MODULE[]', leaves=['TQD_trqTrqSetNormal_MAP_v', 'VBU_L045A_CWP_05_09T_AImode_CM_single', 'Lookup2D_FLOAT32_IEEE', 'Lookup2D_X_FLOAT32_IEEE', 'Scalar_FLOAT32_IEEE', 'TQD_vVehSpd', 'TQD_vSgndSpd_MAP_y', 'TQD_pctAccPedPosFlt', 'TQD_pctAccPdl_MAP_x'], channel=3, download_id=630, upload_id=631, input='/home/n/devel/anting_remote/eos/tspace/candycan/res/download.json', output='/home/n/devel/anting_remote/eos/tspace/candycan/res/output.json')


In [32]:
xcp_calib_from_xcpjson = Get_XCPCalib_From_XCPJSon(args.input)

xcp_data = Generate_Init_XCPData_From_A2L(
    a2l=args.a2l, keys=args.leaves, node_path=args.node_path
)

try:
    XCPData.model_validate(xcp_data)
except ValidationError as exc:
    print(exc)

{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000000...000',
    'value_array_view': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]),
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}

In [38]:
# type(args.channel), type(args.download_id), args.upload_id, args.download, args.diff_flashing

(int, int, 631, True, True)

In [37]:
xcp_data.value = xcp_calib_from_xcpjson.data[0].value
pprint(xcp_data)

xcp_calib = XCPCalib(
    config=XCPConfig(
        channel=args.channel, download=str(args.download_id), upload=str(args.upload_id)
    ),
    data=[xcp_data],
)
# pprint(xcp_calib)

{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000025...344',
    'value_array_view': array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
         4436.   ],
       [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
         4436.   ],
       [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
         4436.   ],
       ...,
       [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
         2034.368],
       [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
         1859.521],
       [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
         1690.249]]),
    'value_length': 4,
    'value_type': 'FLOAT32_IEEE'}
XCPCalib(config=XCPConfig(channel=3, download_can_id='630', upload_can_id='631'), data=[{   'address': '7000aa2a',
    'dim': [14, 17],
    'name': 'TQD_trqTrqSetNormal_MAP_v',
    'type_size': 4,
    'value': '0000000025...344',
    '

In [47]:
npa =  xcp_calib.data[0].value_array_view
npa

array([[    0.   ,  1135.317,  1135.317, ...,  4436.   ,  4436.   ,
         4436.   ],
       [    0.   ,   148.09 ,   148.09 , ...,  4436.   ,  4436.   ,
         4436.   ],
       [-1338.534,  -833.344,  -328.155, ...,  4436.   ,  4436.   ,
         4436.   ],
       ...,
       [-1316.842, -1086.408,  -855.974, ...,  2034.368,  2034.368,
         2034.368],
       [-1141.171,  -926.042,  -710.912, ...,  1859.521,  1859.521,
         1859.521],
       [ -286.008,  -135.973,    14.062, ...,  1690.249,  1690.249,
         1690.249]])

In [112]:
# buffer = [i.hex() for x in npa for i in x]
# # buffer[::-1]
# len(buffer)
# buffer

In [113]:

# buffer = npa.tobytes()

# pprint(buffer), len(buffer)
# xcp_calib.data[0].value, len(xcp_calib.data[0].value)

In [144]:
npb = npa[::-1]
# npb
buffer = [struct.pack("<f", x) for x in np.nditer(npa)]
# buffer
len(buffer)

238

In [120]:
#| export
def npa_to_packed_buffer(a: np.ndarray) -> str:
    """ convert a numpy array to a packed string buffer for flashing
    TODO: implementation as numpy ufunc

    Args:
        a (np.ndarray): input numpy array for flashing

    Returns:
        str: packed string buffer for flashing
    """
    b = [struct.pack("<f", x).hex() for x in np.nditer(a)]
    return ''.join(b)

In [126]:

# buffer = [struct.pack("<f", x).hex() for x in np.nditer(npa)]
# buffer[::-1]
# len(buffer)
# buffer
# data = ''.join(buffer)
data = npa_to_packed_buffer(npa)
test_eq(data, xcp_calib.data[0].value)
# data

In [143]:
# npa.astype(np.float32).tobytes().hex()
buffer = npa.astype(np.float32).tobytes().hex()  ## == npa_to_packed_buffer(npa)
buffer, len(buffer)
test_eq(buffer, xcp_calib.data[0].value)

('0000000025ea8d4425ea8d4425ea8d4425ea8d44db349544b31eb6448c08d74464f2f7441e6e0c45811d2545e3cc3d45467c564585ed834500a08a4500a08a4500a08a4500000000ed161443ed1614434eba8d431a791e44cb4c60443e10914417fab144efe3d244c8cdf34446961245a9452b450bf54345d053754500a08a4500a08a4500a08a451451a7c40b5650c4dd13a4c3b80831434da9f94356254144847c81445c66a2443550c3440d3ae44469cc0a45cb7b23452e2b3c45f3896d4500a08a4500a08a4500a08a45fffc0fc5a4fcd0c44bff81c4c807ccc339db5f4376a00944f8d74744bd0783447f23a244403fc144e2e8ef4442490f45139e2645b5475545abf8814500a08a4500a08a45771f0dc59f6adac451969ac4078435c4ab6d57c3629a9343eb8d18443f2953444ae28644f42fa4447424d044f318fc44b906144539fb3f45b8ef6b4500a08a4500a08a45483e09c51dd0dcc4a923a7c46cee62c40a2befc3dcc943c29338be43be142944e413604485898b44e1c8b4443e08de44cda303452ae32c4586225645e3617f4500a08a45735904c5aa24dac46c96abc45f107ac4e5f31cc4ac5d7fc37928ea4212c3f443ff0b3e44f76e7144b5419f44efcbc5442856ec444eb51c45873f4345c1c96945f1378345efe1fcc4bac2d3c485a3aac44f8481c434ca30c49117bd

In [ ]:
#| export
def flash_xcp(xcp_calib: XCPCalib, data: pd.DataFrame, diff_flashing: bool=False, download: bool=True):
    """Summary
    Flash XCP data to target

    Args:
        xcp_calib (XCPCalib): XCP calibration as template, contains all the meta information except for data
        xcp_data (pd.DataFrame): input XCP data to be flashed, replace the value in xcp_calib
        diff_flashing (bool): Use differential flashing
        download (bool): Download or upload
    
    """
    
    xcp_calib.data = data.astype(np.float32).tobytes().hex()

    if download:
        if diff_flashing:
            raise NotImplementedError("Differential flashing not implemented yet")
        else:
            pass
             
        

    

In [8]:
#| export
if __name__ == "__main__" and "__file__" in globals():  # only run if this file is called directly

    protocol = inquirer.select(
        message="What's the protocol?",
        choices=[
            Choice(value="ccp", name="CCP"),
            Choice(value="xcp", name="XCP"),
        ],
        default="ccp",
    ).execute()

    download = inquirer.confirm(
        message="Downloading(host->target)?",
        confirm_letter="y",
        reject_letter="n",
        default=True,
    ).execute()

    differential_flashing = inquirer.confirm(
        message="Differential Flashing?",
        confirm_letter="y",
        reject_letter="n",
        default=False,
    ).execute()

    a2l_file_path = inquirer.text(
        message="a2l file path",
        validate=EmptyInputValidator(),
        default='/res/vbu_ai.json'
    ).execute()

    # node_path = inquirer.text(
    #     message="node path",
    #     validate=EmptyInputValidator(),
    # 	default=r"/PROJECT/MODULE[]",
    # ).execute()

    # leaves = inquirer.text(
    #     message="leaves",
    #     validate=EmptyInputValidator(),
    # 	default=r"TQD_trqTrqSetNormal_MAP_v, VBU_L045A_CWP_05_09T_AImode_CM_single, Lookup2D_FLOAT32_IEEE, Lookup2D_X_FLOAT32_IEEE, Scalar_FLOAT32_IEEE, TQD_vVehSpd, TQD_vSgndSpd_MAP_y, TQD_pctAccPedPosFlt, TQD_pctAccPdl_MAP_x"
    # ).execute()

    can_channel = inquirer.number(
        message="CAN channel for flashing",
        min_allowed=0,
        max_allowed=32,
        validate=EmptyInputValidator(),
        default=3,
    ).execute()

    download_id = inquirer.number(
        message="CAN ID for downloading",
        min_allowed=0,
        max_allowed=9999,
        validate=EmptyInputValidator(),
        default=630,
    ).execute()

    upload_id = inquirer.number(
        message="CAN ID for uploading",
        min_allowed=0,
        max_allowed=9999,
        validate=EmptyInputValidator(),
        default=631,
    ).execute()

    input_file_path = inquirer.text(
        message="Input file path",
        validate=EmptyInputValidator(),
        default="/res/download.json",
    ).execute()

    output_file_path = inquirer.text(
        message="Output file path",
        validate=EmptyInputValidator(),
        default="/res/output.json",
    ).execute()

    args = get_argparser().parse_args()
    args.protocol = protocol
    args.download = download
    args.diff_flashing = differential_flashing
    # args.a2l = a2l_file_path
    # args.node_path = node_path
    # args.leaves = leaves
    args.channel = can_channel
    args.download_id = download_id
    args.upload_id = upload_id
    args.input = repo.working_dir+input_file_path
    args.output = repo.working_dir+output_file_path
    pprint(args)

    xcp_calib_from_xcpjson = Get_XCPCalib_From_XCPJSon(args.input)
    xcp_data = Generate_Init_XCPData_From_A2L(
        a2l=args.a2l, keys=args.leaves, node_path=args.node_path
    )
    try:
        XCPData.model_validate(xcp_data)
    except ValidationError as exc:
        print(exc)

    xcp_data.value = xcp_calib_from_xcpjson.data[0].value
    pprint(xcp_data)

    xcp_calib = XCPCalib(
        config=XCPConfig(
            channel=args.channel, download=args.download_id, upload=args.upload_id
        ),
        data=[xcp_data],
    )
    pprint(xcp_calib)

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()